## Neural and N-Gram Language Model for CPSC 503 Assignment 2
#### The neural model notebook is modified from Yunjey Choi's Github repository - pytorch-tutorial.
#### Familiarize yourself with pytorch, start with: https://pytorch.org/tutorials/beginner/basics/intro.html

#### The N-gram model notebook is from Josh Loehr's Github repository - ngram-language-model.
#### https://github.com/joshualoehr/ngram-language-model

#### ========================================================================================

### Let's load a number of dependencies:

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
import numpy as np
import os
import math
from collections import defaultdict
from tqdm import tqdm

# check if GPU is available to pytorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Here are two classes needed for the data loading and formating:

In [2]:
class Dictionary(object):
    # use to generate and return the word-to-index (index-to-word) vocabulary dictionary
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def return_dict(self):
        return self.idx2word
        
    def __len__(self):
        return len(self.word2idx)


class Corpus(object):
    # load and prepare the corpus  the language models input format
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, n_gram=2):
        with open(path, 'r') as f:
            tokens = 0;
            sample_list = []
            for line in f:
                # add <start> tokens based on the number of n-grams.
                words = ['<start>'] * (n_gram - 1) + line.split() + ['<end>']
                tokens += len(words)
                sample_list.append(words)
                for word in words: 
                    self.dictionary.add_word(word)  

        # # Read corpus and store the each line (word sequence) into its corresponding index sequence.
        ids_list = [[0]*len(s) for s in sample_list if len(s) > n_gram] 
        with open(path, 'r') as f:
            sample_num = 0
            for line in f:
                token = 0
                words = ['<start>'] * (n_gram - 1) + line.split() + ['<end>']
                if len(words) >= n_gram:
                    for word in words:
                        ids_list[sample_num][token] = self.dictionary.word2idx[word]
                        token += 1
                    sample_num += 1

        # FOR THE NEURAL MODEL Convert the flat index sequences into the n-gram tensors which are ready for neural model.
        for n in range(len(ids_list)):
            flat_ids = ids_list[n]
            ids_list[n] = torch.LongTensor([flat_ids[i:i+n_gram] for i in range(len(flat_ids)-(n_gram - 1))])
        return ids_list

### Hyper-parameters for both Language Models: 

In [3]:
# Trigram model
n_gram = 2

# m_gram is the number of preceding/conditioning words 
m_gram = n_gram - 1

### Load the "Penn Treebank" dataset and split it into train/dev/test

In [4]:
corpus = Corpus()
#ids = corpus.get_data('data/train_mini.txt', n_gram)
ids = corpus.get_data('data/train.txt', n_gram)


# Use 70% for training, 15% for development, and 15% for testing 
n_train = round(len(ids) * .75)
n_dev = round(len(ids) * .15)

train_ids = ids[:n_train]
dev_ids = ids[n_train:n_train + n_dev]
test_ids = ids[n_train + n_dev:]

print(f"Number of sentences: {len(train_ids)} train, {len(dev_ids)} dev, {len(test_ids)} test")
vocab_size = len(corpus.dictionary)
print(f"Vocab size: {vocab_size}")

Number of sentences: 31551 train, 6310 dev, 4207 test
Vocab size: 10001


### The class of count-based language model:

In [5]:
class CountLM(object):
    def __init__(self, vocab_size, x_n, x_m, laplace=1):
        self.vocab_size = vocab_size
        self.laplace = laplace
        
        # Dictionaries for tracking the count of n-grams
        self.n_gram_count = self.count_ngrams(x_n)
        self.m_gram_count = self.count_ngrams(x_m)
    
    def count_ngrams(self, x):
        """
        Populate the dictionary with the number of occurrences of each n-gram
        """
        count_list = defaultdict(int)
        for example in x:
            for n_gram in example.tolist():
                count_list[tuple(n_gram)] += 1 
        return count_list
    
    def compute_mle(self, n_gram):
        """
        Compute the MLE of P(w_n|w_{n−1}, ...) with add-one Laplacian smoothing
        
        Please see chapter 3.5.1 of J&M 3rd Ed. for more information
        """
        n_count = self.n_gram_count[n_gram]
        m_gram = n_gram[:-1]
        m_count = self.m_gram_count[m_gram]
        prob = (n_count + self.laplace) / (m_count + self.laplace * self.vocab_size) 
        return prob


### Train the n-gram model based on MLE

In [6]:
#m_gram_train_ids = corpus.get_data('data/train_mini.txt', m_gram)[:-200]
m_gram_train_ids = corpus.get_data('data/train.txt', m_gram)[:n_train]
n_gram_train_ids = train_ids

# Populate the dictionaries with counts from training corpus
count_model = CountLM(vocab_size, n_gram_train_ids, m_gram_train_ids)

# Compute average perplexity on training set
train_ppl = 0
for i in range(0, len(train_ids)):

    probabilities = list(map(lambda x: count_model.compute_mle(tuple(x)), train_ids[i].tolist()))
    perplexity = np.exp(sum(-np.log(probabilities)) / len(train_ids[i]))
    train_ppl += perplexity
    
print('The average training perplexity for count-based LM: '+str(train_ppl/len(train_ids)))

The average training perplexity for count-based LM: 872.2299472899585


In [7]:
# Compute average perplexity on testing set
test_ppl = 0
for i in range(0, len(test_ids)):

    probabilities = list(map(lambda x: count_model.compute_mle(tuple(x)), test_ids[i].tolist()))
    perplexity = np.exp(sum(-np.log(probabilities)) / len(test_ids[i]))
    print('Perplexity for test sample '+str(i)+' :', perplexity)
    test_ppl += perplexity
    
print('The average testing perplexity for count-based LM: '+str(test_ppl/len(test_ids)))

Perplexity for test sample 0 : 481.93397924150236
Perplexity for test sample 1 : 1791.0646372289625
Perplexity for test sample 2 : 295.4281859753384
Perplexity for test sample 3 : 336.8972086530059
Perplexity for test sample 4 : 188.3063912073697
Perplexity for test sample 5 : 1580.4694541715269
Perplexity for test sample 6 : 908.3518812063347
Perplexity for test sample 7 : 587.567516576945
Perplexity for test sample 8 : 466.1199712311848
Perplexity for test sample 9 : 710.2569626653177
Perplexity for test sample 10 : 3201.2726581592956
Perplexity for test sample 11 : 731.7403780164503
Perplexity for test sample 12 : 902.777634164664
Perplexity for test sample 13 : 2107.650302714663
Perplexity for test sample 14 : 867.2708592238805
Perplexity for test sample 15 : 1282.4375182196761
Perplexity for test sample 16 : 516.5791519391194
Perplexity for test sample 17 : 1637.3482160822728
Perplexity for test sample 18 : 891.4693914511101
Perplexity for test sample 19 : 1841.6775228827773
Perpl

Perplexity for test sample 1532 : 299.9255184957937
Perplexity for test sample 1533 : 827.8076804668558
Perplexity for test sample 1534 : 705.3064289852103
Perplexity for test sample 1535 : 663.849079561899
Perplexity for test sample 1536 : 711.7755184372891
Perplexity for test sample 1537 : 2288.6995492379706
Perplexity for test sample 1538 : 402.1920779200347
Perplexity for test sample 1539 : 374.4837679885483
Perplexity for test sample 1540 : 301.62385188074904
Perplexity for test sample 1541 : 268.5909645429137
Perplexity for test sample 1542 : 480.63808049208467
Perplexity for test sample 1543 : 1405.4494167726066
Perplexity for test sample 1544 : 1233.6836836396255
Perplexity for test sample 1545 : 964.158641688546
Perplexity for test sample 1546 : 2632.0574769360564
Perplexity for test sample 1547 : 736.6576720700164
Perplexity for test sample 1548 : 768.7408603227124
Perplexity for test sample 1549 : 1089.5486884190204
Perplexity for test sample 1550 : 940.3460227573968
Perplex

Perplexity for test sample 1719 : 1610.5562179811188
Perplexity for test sample 1720 : 976.6789786030192
Perplexity for test sample 1721 : 1264.777146984313
Perplexity for test sample 1722 : 1054.3261279681315
Perplexity for test sample 1723 : 2075.2127262142326
Perplexity for test sample 1724 : 2172.520245880961
Perplexity for test sample 1725 : 1736.3325040703128
Perplexity for test sample 1726 : 1043.0389655890112
Perplexity for test sample 1727 : 1215.3594752405668
Perplexity for test sample 1728 : 2867.1898601133594
Perplexity for test sample 1729 : 1132.6420371681181
Perplexity for test sample 1730 : 1155.963543866034
Perplexity for test sample 1731 : 3626.5397264805056
Perplexity for test sample 1732 : 915.0132519441463
Perplexity for test sample 1733 : 1577.6316500799273
Perplexity for test sample 1734 : 1456.797368349581
Perplexity for test sample 1735 : 296.7899207543032
Perplexity for test sample 1736 : 389.04008265227964
Perplexity for test sample 1737 : 2002.0387874570877


Perplexity for test sample 3203 : 1066.9051709594128
Perplexity for test sample 3204 : 213.03255375713894
Perplexity for test sample 3205 : 188.68127805103896
Perplexity for test sample 3206 : 668.5470958926691
Perplexity for test sample 3207 : 49.5250665601804
Perplexity for test sample 3208 : 551.8774206459285
Perplexity for test sample 3209 : 1273.9607351336326
Perplexity for test sample 3210 : 349.93082810695375
Perplexity for test sample 3211 : 94.15180533040544
Perplexity for test sample 3212 : 722.0812189465628
Perplexity for test sample 3213 : 61.70886482940158
Perplexity for test sample 3214 : 1275.3574016951318
Perplexity for test sample 3215 : 249.5157749759525
Perplexity for test sample 3216 : 650.2217474816422
Perplexity for test sample 3217 : 333.3065764268792
Perplexity for test sample 3218 : 1745.8192584724918
Perplexity for test sample 3219 : 2341.2912998599363
Perplexity for test sample 3220 : 819.823846339079
Perplexity for test sample 3221 : 3316.222490281745
Perple

### Hyper-parameters for the neural language model

In [8]:
# FOR THE NEURAL MODEL
embed_size = 128
intermediate_size = 1024
num_epochs = 2
learning_rate = 2e-3

### The class for the neural language model:

In [9]:
class NeuralLM(nn.Module):
    def __init__(self, vocab_size, embed_size, intermediate_size, m_gram):
        super(NeuralLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.intermediate = nn.Linear(m_gram * embed_size, intermediate_size)
        self.final = nn.Linear(intermediate_size, vocab_size)
        
    def forward(self, x):
        x = self.embed(x) # Embed word id(s) to vectors
        conc_emb = x.view(x.size(0), x.size(1)*x.size(2))
        intermediate_output = self.intermediate(conc_emb) # one layer of MLP
        intermediate_output = F.relu(intermediate_output) # ReLU non-linear function
        final_out = self.final(intermediate_output) # Map to the vocabulary size output
        return final_out

In [10]:
neural_model = NeuralLM(vocab_size, embed_size, intermediate_size, m_gram).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(neural_model.parameters(), lr=learning_rate, momentum=0.9)

### Train the neural model

In [11]:
# Reduce batch size if you are running out of memory
batch_size = 64
training_data = torch.cat(train_ids, dim=0)
neural_model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for i in tqdm(range(0, len(training_data), batch_size)):
        batch = training_data[i:i + batch_size]
        inputs = batch[:, 0:n_gram-1].to(device)
        targets = batch[:, n_gram-1:].to(device)
        
        # Forward pass
        outputs = neural_model(inputs)
        loss = criterion(outputs, targets.reshape(-1))
        total_loss += loss.item();
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    # Calculate the performance (perplexity) of the current trained model on dev set.
    total_ppl = 0
    for i in range(0, len(dev_ids)):
        dev_inputs = dev_ids[i][:, 0:n_gram-1].to(device)
        dev_targets = dev_ids[i][:, n_gram-1:].to(device)
        dev_outputs = neural_model(dev_inputs)
        ce = criterion(dev_outputs, dev_targets.reshape(-1))
        total_ppl += np.exp(ce.item());
    
    print ('Epoch [{}/{}], Training Loss: {:.4f}, Dev Perplexity: {:5.2f}'
        .format(epoch + 1, num_epochs, total_loss/len(train_ids), total_ppl/len(dev_ids)))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10910/10910 [00:25<00:00, 426.50it/s]


Epoch [1/2], Training Loss: 2.1698, Dev Perplexity: 539.77


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10910/10910 [00:25<00:00, 420.16it/s]


Epoch [2/2], Training Loss: 2.0145, Dev Perplexity: 473.09


### Save the trained model

In [12]:
torch.save(neural_model, 'model.ckpt')

### Neural Model testing

In [13]:
neural_model = torch.load('model.ckpt')
neural_model.eval()
test_ppl = 0
with torch.no_grad():
    for i in range(0, len(test_ids)):
        inputs = test_ids[i][:, 0:n_gram-1].to(device)
        gold = test_ids[i][:, n_gram-1:].to(device)
        output = neural_model(inputs)
        cross_entropy = criterion(output, gold.reshape(-1)).item()
        print('Perplexity for test sample '+str(i)+' :', np.exp(cross_entropy))
        test_ppl += np.exp(cross_entropy)
    print('The average testing perplexity for neural LM: '+str(test_ppl/len(test_ids)))

Perplexity for test sample 0 : 148.00940652047356
Perplexity for test sample 1 : 973.6055487290955
Perplexity for test sample 2 : 176.30954188459538
Perplexity for test sample 3 : 150.6752268866717
Perplexity for test sample 4 : 82.06285557872266
Perplexity for test sample 5 : 503.8194836996781
Perplexity for test sample 6 : 464.4355918638287
Perplexity for test sample 7 : 333.8638644190184
Perplexity for test sample 8 : 255.27581042750313
Perplexity for test sample 9 : 284.64254071234456
Perplexity for test sample 10 : 1086.9737777951377
Perplexity for test sample 11 : 297.2834128976365
Perplexity for test sample 12 : 285.2025622936094
Perplexity for test sample 13 : 511.6607657402606
Perplexity for test sample 14 : 250.838633343745
Perplexity for test sample 15 : 521.7374428519541
Perplexity for test sample 16 : 138.4858304260492
Perplexity for test sample 17 : 740.0227064032316
Perplexity for test sample 18 : 377.21257050442955
Perplexity for test sample 19 : 550.073769098956
Perple

Perplexity for test sample 168 : 138.76259595751
Perplexity for test sample 169 : 486.19614315085323
Perplexity for test sample 170 : 254.18117844126462
Perplexity for test sample 171 : 412.88182387869057
Perplexity for test sample 172 : 173.8420030424625
Perplexity for test sample 173 : 247.17829082211568
Perplexity for test sample 174 : 168.20060698836426
Perplexity for test sample 175 : 246.3332408790106
Perplexity for test sample 176 : 197.57627380515564
Perplexity for test sample 177 : 612.201993357633
Perplexity for test sample 178 : 217.62084944734283
Perplexity for test sample 179 : 355.99359488711525
Perplexity for test sample 180 : 416.6410801024257
Perplexity for test sample 181 : 140.59270055220978
Perplexity for test sample 182 : 119.44687635205867
Perplexity for test sample 183 : 193.5073570513612
Perplexity for test sample 184 : 804.8824480362947
Perplexity for test sample 185 : 610.7508268876735
Perplexity for test sample 186 : 434.3779487261097
Perplexity for test samp

Perplexity for test sample 407 : 1128.1862861337042
Perplexity for test sample 408 : 550.725960024268
Perplexity for test sample 409 : 966.4721260998075
Perplexity for test sample 410 : 427.5126485859433
Perplexity for test sample 411 : 741.033310704468
Perplexity for test sample 412 : 441.4608411690195
Perplexity for test sample 413 : 190.20440831194205
Perplexity for test sample 414 : 243.4526914302777
Perplexity for test sample 415 : 860.4695573396959
Perplexity for test sample 416 : 2082.200435697593
Perplexity for test sample 417 : 698.0390131018432
Perplexity for test sample 418 : 144.68746413463148
Perplexity for test sample 419 : 106.79889197723598
Perplexity for test sample 420 : 546.7459024711665
Perplexity for test sample 421 : 138.10105063663667
Perplexity for test sample 422 : 1162.6602270003582
Perplexity for test sample 423 : 379.16414355092627
Perplexity for test sample 424 : 225.94258840184537
Perplexity for test sample 425 : 750.8570920738292
Perplexity for test sampl

Perplexity for test sample 598 : 562.5673994703226
Perplexity for test sample 599 : 507.89884649130664
Perplexity for test sample 600 : 611.9153943765408
Perplexity for test sample 601 : 1871.363142712741
Perplexity for test sample 602 : 979.4372278443545
Perplexity for test sample 603 : 832.2655922441054
Perplexity for test sample 604 : 492.81192068257866
Perplexity for test sample 605 : 1734.942008142064
Perplexity for test sample 606 : 303.0769924197994
Perplexity for test sample 607 : 626.2628816635785
Perplexity for test sample 608 : 916.5457220769966
Perplexity for test sample 609 : 1233.7458775311
Perplexity for test sample 610 : 839.5225903674096
Perplexity for test sample 611 : 876.8853441601628
Perplexity for test sample 612 : 1060.9154792749837
Perplexity for test sample 613 : 190.88639619957746
Perplexity for test sample 614 : 618.4931694713821
Perplexity for test sample 615 : 1048.909992537846
Perplexity for test sample 616 : 336.53133114878455
Perplexity for test sample 6

Perplexity for test sample 829 : 856.3089101962157
Perplexity for test sample 830 : 136.63655848605978
Perplexity for test sample 831 : 31.637030904694342
Perplexity for test sample 832 : 366.55902025726783
Perplexity for test sample 833 : 657.9905558828527
Perplexity for test sample 834 : 576.2975610987806
Perplexity for test sample 835 : 368.92839347281813
Perplexity for test sample 836 : 375.7781516848374
Perplexity for test sample 837 : 193.06597300453498
Perplexity for test sample 838 : 547.5032640794936
Perplexity for test sample 839 : 556.9066930299297
Perplexity for test sample 840 : 109.56431686495883
Perplexity for test sample 841 : 205.92563846060943
Perplexity for test sample 842 : 216.24930717375796
Perplexity for test sample 843 : 494.26937107183636
Perplexity for test sample 844 : 1035.488301816344
Perplexity for test sample 845 : 226.65142773755656
Perplexity for test sample 846 : 427.56361510283597
Perplexity for test sample 847 : 363.96267980826116
Perplexity for test

...

Perplexity for test sample 3110 : 30.559914865107245
Perplexity for test sample 3111 : 850.5542817893878
Perplexity for test sample 3112 : 785.6235079909582
Perplexity for test sample 3113 : 317.851799739588
Perplexity for test sample 3114 : 642.4632520646602
Perplexity for test sample 3115 : 49.53412373159585
Perplexity for test sample 3116 : 1423.986607095454
Perplexity for test sample 3117 : 110.7927821037333
Perplexity for test sample 3118 : 49.53412373159585
Perplexity for test sample 3119 : 357.41281849622567
Perplexity for test sample 3120 : 57.317250103992556
Perplexity for test sample 3121 : 433.47374375578624
Perplexity for test sample 3122 : 57.317250103992556
Perplexity for test sample 3123 : 600.2092286634064
Perplexity for test sample 3124 : 41.10268767970916
Perplexity for test sample 3125 : 245.28677955589032
Perplexity for test sample 3126 : 29.2979404973042
Perplexity for test sample 3127 : 354.91409097628434
Perplexity for test sample 3128 : 194.66530888067706
Perple

Perplexity for test sample 3318 : 449.0847352244739
Perplexity for test sample 3319 : 241.37655083749564
Perplexity for test sample 3320 : 608.0022978675455
Perplexity for test sample 3321 : 257.81401795973017
Perplexity for test sample 3322 : 289.7097352994038
Perplexity for test sample 3323 : 59.07147941761611
Perplexity for test sample 3324 : 148.17259844557267
Perplexity for test sample 3325 : 504.19271436489834
Perplexity for test sample 3326 : 285.0875335750039
Perplexity for test sample 3327 : 1121.713884378927
Perplexity for test sample 3328 : 392.4556563810025
Perplexity for test sample 3329 : 4865.7675110621085
Perplexity for test sample 3330 : 473.46028566915305
Perplexity for test sample 3331 : 1205.4334353164002
Perplexity for test sample 3332 : 425.50594641308777
Perplexity for test sample 3333 : 350.0922654736913
Perplexity for test sample 3334 : 317.14039992263565
Perplexity for test sample 3335 : 816.0611012222214
Perplexity for test sample 3336 : 1234.6451211908802
Pe

Perplexity for test sample 3490 : 377.1595128900472
Perplexity for test sample 3491 : 366.1813212449831
Perplexity for test sample 3492 : 342.4812551017757
Perplexity for test sample 3493 : 383.80670202851263
Perplexity for test sample 3494 : 277.31843095392554
Perplexity for test sample 3495 : 239.11131288013445
Perplexity for test sample 3496 : 543.8275238988849
Perplexity for test sample 3497 : 494.5294011079455
Perplexity for test sample 3498 : 437.94976240172554
Perplexity for test sample 3499 : 1367.192675274508
Perplexity for test sample 3500 : 483.9933703282232
Perplexity for test sample 3501 : 566.9388478360371
Perplexity for test sample 3502 : 327.41806190510107
Perplexity for test sample 3503 : 282.8680206120309
Perplexity for test sample 3504 : 225.50000451966557
Perplexity for test sample 3505 : 203.20477807647214
Perplexity for test sample 3506 : 105.4518179368834
Perplexity for test sample 3507 : 45.29899622986673
Perplexity for test sample 3508 : 54.3948361734495
Perple

Perplexity for test sample 3760 : 1694.3658820941903
Perplexity for test sample 3761 : 29.960595406870677
Perplexity for test sample 3762 : 118.1783990601067
Perplexity for test sample 3763 : 83.49271545321965
Perplexity for test sample 3764 : 130.9592277123395
Perplexity for test sample 3765 : 134.93398152520973
Perplexity for test sample 3766 : 144.14344979475877
Perplexity for test sample 3767 : 176.15676714302126
Perplexity for test sample 3768 : 189.5711333570083
Perplexity for test sample 3769 : 63.89474070546725
Perplexity for test sample 3770 : 171.73976864567095
Perplexity for test sample 3771 : 202.23581427221134
Perplexity for test sample 3772 : 292.7462024117967
Perplexity for test sample 3773 : 15.152707455484503
Perplexity for test sample 3774 : 134.9816670276189
Perplexity for test sample 3775 : 125.67098709934342
Perplexity for test sample 3776 : 160.30593199011767
Perplexity for test sample 3777 : 14.070748154369104
Perplexity for test sample 3778 : 277.784953879027
Pe

Perplexity for test sample 3957 : 1010.5423587086101
Perplexity for test sample 3958 : 242.6440962118028
Perplexity for test sample 3959 : 511.0633665438468
Perplexity for test sample 3960 : 354.79953654206656
Perplexity for test sample 3961 : 1308.9080871495073
Perplexity for test sample 3962 : 270.10304118215936
Perplexity for test sample 3963 : 115.54025612444269
Perplexity for test sample 3964 : 789.6406199094724
Perplexity for test sample 3965 : 666.4512230885301
Perplexity for test sample 3966 : 1419.266502588094
Perplexity for test sample 3967 : 53.98355207844692
Perplexity for test sample 3968 : 537.8163100732276
Perplexity for test sample 3969 : 805.2659534337698
Perplexity for test sample 3970 : 920.7013419777815
Perplexity for test sample 3971 : 883.4468394409223
Perplexity for test sample 3972 : 2838.16758424287
Perplexity for test sample 3973 : 100.38526813644013
Perplexity for test sample 3974 : 441.44400109558353
Perplexity for test sample 3975 : 312.1543723166045
Perple